Importe os dados (ver slide 5)
A variável ano_mes ainda não está como estrutura tempora. Transforme ela para formato temporal com a função yearmonth e convirta seu dataset para um dataset em formato de série temporal (ver slide 5)
Utilize a função stretch_tsibble() para criar vários conjuntos de treinamento. Defina .init = 50 e .step = 1. (ver slide 22)
Ajuste um modelo SNAIVE() (SNAIVE(temp_media~lag(12))), faça a previsão 
h
=
3
 passos à frente (para cada conjunto de treinamento) e finalmente avalie o desempenho do modelo com a função accuracy(). (ver slide 22)
Qual modelo teve um melhor desempenho fora da amostra, o modelo SNAIVE ou o modelo desenvolvido na aula? (compare o RMSE e o MAE).

In [ ]:
install.packages("fpp3")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘numDeriv’, ‘warp’, ‘BH’, ‘distributional’, ‘progressr’, ‘slider’, ‘anytime’, ‘fable’, ‘fabletools’, ‘feasts’, ‘tsibble’, ‘tsibbledata’, ‘urca’


── Attaching packages ──────────────────────────────────────────── fpp3 0.4.0 ──

✔ tibble      3.1.6     ✔ tsibble     1.1.1
✔ dplyr       1.0.7     ✔ tsibbledata 0.4.0
✔ tidyr       1.1.4     ✔ feasts      0.2.2
✔ lubridate   1.8.0     ✔ fable       0.3.1

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Conflicts ───────────────────────────────────────────────── fpp3_conflicts ──
✖ lubridate::date()    masks base::date()
✖ dplyr::filter()      masks stats::filter()
✖ tsibble::intersect() masks base::intersect()
✖ tsibble::interval()  masks lubridate::interval()
✖ dplyr::lag()         masks stats::lag()
✖ tsibble::setdiff()   masks base::setdiff()
✖ tsibble::union()     m

In [ ]:
library(ggplot2)
library(slider)
library(fpp3)
library(tsibble)

In [ ]:
df <- read.csv("https://raw.githubusercontent.com/ctruciosm/ctruciosm.github.io/master/datasets/lajeado_rs.csv", sep = ";")
glimpse(df)

Rows: 79
Columns: 4
$ ano_mes    <chr> "2015-01", "2015-02", "2015-03", "2015-04", "2015-05", "201…
$ temp_media <dbl> 25.6, 24.8, 24.2, 21.2, 18.5, 16.2, 15.9, 21.0, 17.8, 19.4,…
$ temp_max   <dbl> 39.0, 35.1, 34.7, 32.4, 30.8, 29.7, 29.2, 34.2, 30.0, 35.0,…
$ temp_min   <dbl> 17.2, 17.5, 15.7, 13.4, 8.4, 4.7, 6.5, 9.5, 4.5, 9.0, 12.5,…


In [ ]:
df <- df %>% 
  mutate(ano_mes = yearmonth(ano_mes)) %>% 
  select(ano_mes, temp_media) %>% 
  as_tsibble(index = ano_mes)
glimpse(df)

Rows: 79
Columns: 2
$ ano_mes    <mth> 2015 Jan, 2015 Feb, 2015 Mar, 2015 Apr, 2015 May, 2015 Jun,…
$ temp_media <dbl> 25.6, 24.8, 24.2, 21.2, 18.5, 16.2, 15.9, 21.0, 17.8, 19.4,…


In [ ]:
df2 <- df %>% 
  stretch_tsibble(.init = 3, .step = 1) 
glimpse(df2)

Rows: 3,157
Columns: 3
Key: .id [77]
$ ano_mes    <mth> 2015 Jan, 2015 Feb, 2015 Mar, 2015 Jan, 2015 Feb, 2015 Mar,…
$ temp_media <dbl> 25.6, 24.8, 24.2, 25.6, 24.8, 24.2, 21.2, 25.6, 24.8, 24.2,…
$ .id        <int> 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5,…


In [ ]:
df2 %>%
  model(RW(temp_media ~ drift())) %>%
  forecast(h = 3) %>%
  group_by(.id) %>%
  mutate(h = row_number()) %>%
  ungroup() %>% 
  accuracy(df, by = c("h", ".model"))

Warning message:
“The future dataset is incomplete, incomplete out-of-sample data will be treated as missing. 
3 observations are missing between 2021 Aug and 2021 Oct”


h,.model,.type,ME,RMSE,MAE,MPE,MAPE,MASE,RMSSE,ACF1
<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,RW(temp_media ~ drift()),Test,0.1275780,2.637200,2.100578,-0.1784376,10.91851,1.534774,1.483226,0.3068485
2,RW(temp_media ~ drift()),Test,0.2988178,4.390662,3.764133,-0.8372524,19.56643,2.750239,2.469416,0.6907249
3,RW(temp_media ~ drift()),Test,0.5241629,6.017084,5.362942,-1.6593484,27.85476,3.918398,3.384155,0.7608784


##### Tanto o MAE quanto o RMSE expressam o erro médio de previsão do modelo em unidades da variável de interesse. Logo, o modelo em aula foi mais eficaz, pois pontuou menos na unidade de medida da variável de interesse.